In [27]:
import requests
from bs4 import BeautifulSoup
import re

In [28]:
# yahoo 電影的網址後面都是那一部電影的電影ID，所以分析後來後，能方便使用
# https://movies.yahoo.com.tw/movieinfo_main/%E7%84%A1%E5%90%8D%E5%BC%92-nobody-11126
# 無名弒 的電影 ID 就是 11126

def get_movie_id(url):
    try:
        movie_id = url.split('.html')[0].split('-')[-1]
    except:
        movie_id = url
    return movie_id

In [29]:
url = 'https://movies.yahoo.com.tw/movieinfo_main/%E7%84%A1%E5%90%8D%E5%BC%92-nobody-11126'
print(get_movie_id(url))

11126


In [30]:
# '\d+' 表示一個或多個數字
def get_date(date_str):
    pattern = '\d+-\d+-\d+'
    match = re.search(pattern, date_str)
    if match is None:
        return date_str
    else:
        return match.group(0)

In [31]:
print(get_date('02-04-2021'))

02-04-2021


In [35]:
def get_movies(dom):
    soup = BeautifulSoup(dom,'html5lib')
    movies = []
    rows = soup.find_all('div','release_info_text')
    print(rows)
    for row in rows:
        movie = dict()
        print(row.find('div','relaease_movie_name').a.text.strip())
    return 

In [38]:
def get_web_page(url):
    resp = requests.get(
    url = url
    )
    if resp.status_code != 200:
        print('Invalid url:',resp.url)
    else:
        return resp.text

movie_thisweek_url = 'https://movies.yahoo.com.tw/movie_thisweek.html'
print(get_web_page(movie_thisweek_url))

<!DOCTYPE html>
<html lang="zh_TW">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1, user-minimum-scale=1, maximum-scale=1">
  <meta http-equiv="content-type" content="text/html; charset=utf-8">
  <meta property="fb:app_id" content="501887343352051">
  <meta property="og:site_name" content="Yahoo奇摩電影">
    <title>本週新片 - Yahoo奇摩電影</title>
<meta name="twitter:title" content="本週新片 - Yahoo奇摩電影">
  <meta property="og:title" content="本週新片 - Yahoo奇摩電影">

<link rel="canonical" href="https://movies.yahoo.com.tw/movie_thisweek.html">
<meta property="og:url" content="https://movies.yahoo.com.tw/movie_thisweek.html">

<meta name="twitter:description" content="Yahoo奇摩電影提供本週的最新電影資訊，查詢時刻表、看最新預告片，讓你電影資訊不漏接">
<meta name="description" content="Yahoo奇摩電影提供本週的最新電影資訊，查詢時刻表、看最新預告片，讓你電影資訊不漏接">

  <meta property="og:description" content="Yahoo奇摩電影提供本週的最新電影資訊，查詢時刻表、看最新預告片，讓你電影資訊不漏接">

<meta name="twitter:image" content="https://movies.yahoo.com.tw/build/imag